In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding
from keras.utils import to_categorical

# Sample data for character prediction
char_data = "hello world"

# Sample data for word prediction
word_data = "Hello world. I am learning deep learning. I love neural networks."

# Character prediction
chars = sorted(list(set(char_data)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

# Prepare the dataset for character prediction
seq_length_char = 3
dataX_char, dataY_char = [], []
for i in range(len(char_data) - seq_length_char):
    seq_in = char_data[i:i + seq_length_char]
    seq_out = char_data[i + seq_length_char]
    dataX_char.append([char_to_int[char] for char in seq_in])
    dataY_char.append(char_to_int[seq_out])

X_char = np.reshape(dataX_char, (len(dataX_char), seq_length_char, 1))
X_char = X_char / float(len(chars))
y_char = to_categorical(dataY_char)

# Word prediction
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer()
tokenizer.fit_on_texts([word_data])
word_total_words = len(tokenizer.word_index) + 1

# Convert text to sequence of tokens
input_sequences = []
for line in word_data.split('.'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

# Pad sequences
max_sequence_len_word = max([len(x) for x in input_sequences])
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_len_word, padding='pre')

X_word, y_word = input_sequences[:,:-1], input_sequences[:,-1]
y_word = to_categorical(y_word, num_classes=word_total_words)

# Define the LSTM model for character prediction
model_char = Sequential()
model_char.add(LSTM(32, input_shape=(X_char.shape[1], X_char.shape[2])))
model_char.add(Dense(y_char.shape[1], activation='softmax'))
model_char.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_char.summary()

# Train the character prediction model
model_char.fit(X_char, y_char, epochs=500, batch_size=1, verbose=2)

# Define the LSTM model for word prediction
model_word = Sequential()
model_word.add(Embedding(word_total_words, 100, input_length=max_sequence_len_word-1))
model_word.add(LSTM(150, return_sequences=True))
model_word.add(LSTM(100))
model_word.add(Dense(word_total_words, activation='softmax'))
model_word.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_word.summary()

# Train the word prediction model
model_word.fit(X_word, y_word, epochs=100, verbose=2)

# Character prediction function
def predict_next_char(model, chars, char_to_int, seq_length, seed_text):
    pattern = [char_to_int[char] for char in seed_text]
    for i in range(seq_length):
        x = np.reshape(pattern, (1, len(pattern), 1))
        x = x / float(len(chars))
        prediction = model.predict(x, verbose=0)
        index = np.argmax(prediction)
        result = chars[index]
        seq_in = [chars[value] for value in pattern]
        print(seq_in, "->", result)
        pattern.append(index)
        pattern = pattern[1:]

# Word prediction function
def predict_next_word(model, tokenizer, text):
    sequence = tokenizer.texts_to_sequences([text])
    sequence = pad_sequences(sequence, maxlen=max_sequence_len_word-1, padding='pre')
    prediction = model.predict(sequence, verbose=0)
    predicted_index = np.argmax(prediction, axis=-1)
    predicted_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted_index:
            predicted_word = word
            break
    return predicted_word

# Test character prediction
print("Character prediction:")
predict_next_char(model_char, chars, char_to_int, seq_length_char, "wor")

# Test word prediction
print("Word prediction:")
print(predict_next_word(model_word, tokenizer, "I am learning"))

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 32)                4352      
                                                                 
 dense (Dense)               (None, 8)                 264       
                                                                 
Total params: 4616 (18.03 KB)
Trainable params: 4616 (18.03 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/500
8/8 - 2s - loss: 2.0879 - accuracy: 0.1250 - 2s/epoch - 274ms/step
Epoch 2/500
8/8 - 0s - loss: 2.0750 - accuracy: 0.2500 - 32ms/epoch - 4ms/step
Epoch 3/500
8/8 - 0s - loss: 2.0654 - accuracy: 0.2500 - 40ms/epoch - 5ms/step
Epoch 4/500
8/8 - 0s - loss: 2.0572 - accuracy: 0.2500 - 32ms/epoch - 4ms/step
Epoch 5/500
8/8 - 0s - loss: 2.0498 - accuracy: 0.2500 - 31ms/epoch - 4ms/step
Epoch 6/500
8/8 - 0s - 